In [ ]:
function Franke(x,y)
    p1 = 3/4*ℯ^(-(9x-2)^2/4-(9y-2)^2/4)
    p2 = 3/4*ℯ^(-(9x+1)^2/49-(9y+1)^2/10)
    p3 = 1/2*ℯ^(-(9x-7)^2/4-(9y-3)^2/4)
    p4 = -1/5*ℯ^(-(9x-4)^2-(9y-7)^2)
    return p1+p2+p3+p4
end
"""
    SplitData2D(xData, yData, zData, train_size)

Splits data with given split: train_size


# Input
 - xData::Array:        Data in X  
 - yData::Array:        Data in Y
 - zData::Array:        Data in Z
 - train_size::Float64: Proportion of data to be given as trainingdata

"""
function SplitData2D(xData::Array, yData::Array, zData::Array, train_size::Float64)
    # Assumes xData and yData are vectors, and zData is a matrix representing the grid

    # Flatten the Z matrix and corresponding X, Y arrays
    xGrid = repeat(xData, inner=length(yData))
    yGrid = repeat(yData, outer=length(xData))
    zVector = vec(zData)

    # Number of total samples
    N = length(zVector)
    
    # Determine number of training samples
    NumTrain = Int(round(N * train_size))
    
    # Randomly sample indices for training data
    train_indices = sample(1:N, NumTrain; replace=false)
    
    # Determine the test indices
    all_indices = collect(1:N)
    test_indices = [i for i in all_indices if i ∉ train_indices]
    
    # Create training and testing datasets
    xTrain = xGrid[train_indices]
    yTrain = yGrid[train_indices]
    zTrain = zVector[train_indices]
    
    xTest = xGrid[test_indices]
    yTest = yGrid[test_indices]
    zTest = zVector[test_indices]
    
    return xTrain, yTrain, zTrain, xTest, yTest, zTest
end
"""
    DataProjector(x, y, z, numDegrees;train, normalize)

This function Projects the Data from the type we generate to the type we need for Regression.


# Input
 - x:        Data in X  
 - y:        Data in Y
 - z:        Data in Z
 - numDegrees: The number of orders of the polynomial
 - train: Whether or not the data should be handeled as training data
 - normalize: Whether or not the DesignMatrix should be normalized
"""
function DataProjector(x, y, z, numDegrees;train=true, normalize=false)

    if !train
        #We flatten the data
        numSamples = length(x)*length(y)
        xData = repeat(x, inner=length(y))  # Repeat xData 
        yData = repeat(y, outer=length(x))  # Repeat yData 
        zData = vec(z) #Flattened Z data
    else
        numSamples = length(z)
        xData=x
        yData=y
        zData=z
    end
     #NxN
    numFeatures = (numDegrees + 1) * (numDegrees + 2) ÷ 2  # Number of polynomial terms
    DesignMatrix = zeros(numSamples, numFeatures)
    column=1
    for i in 0:numDegrees
        for j in 0:(numDegrees-i)
            DesignMatrix[:,column] .= (xData .^ i) .* (yData .^ j)
            column+=1
        end
    end
    if normalize
        col_std = zeros(column-1)
        col_mean = zeros(column-1)
        for j in 1:(column-1)
            col_mean[j] = mean(DesignMatrix[:, j])
            col_std[j] = std(DesignMatrix[:, j])

            # Avoid dividing by zero if the standard deviation is zero (constant column)
            if col_std[j] != 0
                DesignMatrix[:, j] = (DesignMatrix[:, j] .- col_mean[j]) ./ col_std[j]
            else
                DesignMatrix[:, j] = DesignMatrix[:, j]  # No standardization if std is 0
            end
        end
    end
    if train
        if !normalize
            return DesignMatrix
        else
            return DesignMatrix, col_std, col_mean
        end
    else
        if !normalize
            return DesignMatrix, zData
        else
            return DesignMatrix, zData, col_std, col_mean
        end
    end
end

function LassoReg(DesignMatrix, zData, γ)
    (T, K) = (size(DesignMatrix, 1), size(DesignMatrix, 2))
    Q = DesignMatrix'DesignMatrix #/ T
    c = DesignMatrix'zData #/ T                      #c'b = Y'X*b

    b = Variable(K)              #define variables to optimize over
    L1 = quadform(b, Q)            #b'Q*b
    L2 = dot(c, b)                 #c'b
    L3 = norm(b, 1)                #sum(|b|)

    # u'u/T + γ*sum(|b|) where u = Y-Xb
    problem = minimize(L1 - 2 * L2 + γ * L3)

    solve!(problem, SCS.Optimizer; silent = true)
    problem.status == Convex.MOI.OPTIMAL ? beta = vec(Convex.evaluate(b)) : beta = NaN

    return beta
end
"""
    Regression(DesignMatrix, zData, Method; λ)

Preforms a Regression with a DesignMatrix, zData and a given method


# Input
 - DesignMatrix: The designmatrix for the regression
 - zData:        The z data for the regression
 - Method:       The method, either; OLS, Ridge or Lasso
 - λ:            Parameter for Ridge and Lasso regression
"""
function Regression(DesignMatrix, zData, Method; λ=0)
    if Method=="OLS"
        Hessian = Transpose(DesignMatrix)*DesignMatrix
        beta = inv(Hessian)*Transpose(DesignMatrix)*zData
        
    elseif Method=="Ridge"
        Hessian = Transpose(DesignMatrix)*DesignMatrix
        beta = inv(Hessian+λ*I)*Transpose(DesignMatrix)*zData
        
    elseif Method=="Lasso"
        beta = LassoReg(DesignMatrix, zData, λ)
        
    else
        @warn "Method Undefined" 
        println("Expected: OLS, Lasso or Ridge, but got: ", Method)
        return
    end
    return beta
end

function MSE(DesignMatrix, z, beta; λ=0, γ=0)
    N = length(z)
    MSE = 1/N*sum((z .- DesignMatrix*beta).^2)+λ*(Transpose(beta)*beta) + γ*sum(abs.(beta))
    return MSE
end

function R2(DesignMatrix, zData, beta)
    N = length(zData)
    z_avg = 1/N*sum(zData)
    R2 = 1- sum((zData.-DesignMatrix*beta).^2)/sum((zData .- z_avg).^2)
    return R2
end

"""
    Bootstrap(xData, yData, zData, numDegrees::Int64, numBootstraps::Int64, Method; λ=0)

Perform bootstrap resampling on the input data to estimate regression coefficients using a specified method (e.g., OLS, Ridge, or Lasso). Generates multiple bootstrap samples from the data, performs regression on each sample, and returns the corresponding coefficients.

# Arguments
- `xData::Array{Float64}`: x-coordinates of the input data.
- `yData::Array{Float64}`: y-coordinates of the input data.
- `zData::Array{Float64}`: Target values for regression (dependent variable).
- `numDegrees::Int64`: Degree of the polynomial used to create the design matrix. This defines the model complexity.
- `numBootstraps::Int64`: Number of bootstrap samples to generate.
- `Method::Symbol`: Regression method to use. Options include:
    - `:OLS` for Ordinary Least Squares.
    - `:Ridge` for Ridge regression.
    - `:Lasso` for Lasso regression.
- `λ::Float64` (optional): Regularization parameter, used for Ridge or Lasso regression. Default is `0`.

# Returns
- `betas::Matrix{Float64}`: A matrix of size `(numCoefficients, numBootstraps)`, where each column contains the regression coefficients for a bootstrap sample.

# Details
1. **Bootstrap Sampling**: The function resamples the original data with replacement to create new datasets for each bootstrap iteration.
2. **Design Matrix**: For each bootstrap sample, the design matrix is constructed using the `DataProjector` function, based on the specified polynomial degree.
3. **Regression**: The specified regression method is applied on the bootstrap sample using the `Regression` function. If the method is Ridge or Lasso, regularization is controlled by the `λ` parameter.
4. **Coefficient Storage**: The regression coefficients for each bootstrap sample are stored in the `betas` matrix.

# Example
```julia
xData = rand(100)
yData = rand(100)
zData = rand(100)
numDegrees = 5
numBootstraps = 100
Method = :OLS

betas = Bootstrap(xData, yData, zData, numDegrees, numBootstraps, Method)
"""
function Bootstrap(xData, yData, zData, numDegrees::Int64, numBootstraps::Int64, Method; λ=0)
    N = length(zData)
    betas = zeros((numDegrees + 1) * (numDegrees + 2) ÷ 2,numBootstraps)
    
    for b in 1:numBootstraps
        # Sample indices with replacement to create a bootstrap sample
        bootstrap_indices = sample(1:N, N; replace=true)
        
        # Create bootstrap sample for xTrain, yTrain, and zTrain
        xBootstrap = xData[bootstrap_indices]
        yBootstrap = yData[bootstrap_indices]
        zBootstrap = zData[bootstrap_indices]

        # Create the design matrix for the bootstrap sample
        DensityMatrixB_strap = DataProjector(xBootstrap, yBootstrap, zBootstrap, numDegrees)
        
        # Perform regression on the bootstrap sample
        betaVals = Regression(DensityMatrixB_strap, zBootstrap, Method, λ=λ)
        
        # Store the regression coefficients
        betas[:,b] .= betaVals
    end

    return betas
end

"""
    CrossValidation(X, Y, Z, k_folds::Int64, degrees::Int64, Method; lambda=0)

Performs k-fold cross-validation to evaluate a regression model on the provided dataset. The function splits the data into `k_folds`, fits a model on the training data, and evaluates it on the test data for each fold, returning the mean error across all folds.

# Arguments
- `X::Array{Float64}`: The x-coordinates of the input data (for grid).
- `Y::Array{Float64}`: The y-coordinates of the input data (for grid).
- `Z::Array{Float64}`: The z-values (target) of the dataset.
- `k_folds::Int64`: The number of folds to use in cross-validation. The data is split into `k_folds` parts, where each part is used once as the test set.
- `degrees::Int64`: The degree of the polynomial used to construct the design matrix.
- `Method::Symbol`: The regression method to use. Can be:
    - `:OLS` for Ordinary Least Squares.
    - `:Ridge` for Ridge regression.
    - `:Lasso` for Lasso regression.
- `lambda::Float64` (optional): The regularization parameter for Ridge or Lasso regression. Default is `0` (no regularization).

# Returns
- `mean_error::Float64`: The mean error (MSE) across all the k-folds of cross-validation.

# Details
1. **Data Processing**: The input data `X`, `Y`, and `Z` are flattened and reshaped to prepare for the regression model. A design matrix is constructed using the `DataProjector` function, mapping the inputs into a higher-dimensional polynomial space defined by `degrees`.
2. **Shuffling and Splitting**: The data is shuffled randomly to remove any bias from its original ordering. The shuffled data is split into `k_folds` parts for cross-validation.
3. **Cross-Validation Loop**: For each fold:
    - The training set consists of all data except the current fold, and the test set consists of the current fold.
    - A regression model is fitted on the training data.
    - The model is evaluated on the test data, and the error (MSE) is calculated.
4. **Error Calculation**: The mean squared error (MSE) is computed for each fold, and the average error across all folds is returned as the overall performance metric.

# Example
```julia
X = rand(10)
Y = rand(10)
Z = rand(10, 10)
k_folds = 5
degrees = 3
Method = :Ridge

meanerror = CrossValidation(X, Y, Z, k_folds, degrees, Method, lambda=0.1)

"""
function CrossValidation(X, Y, Z, k_folds::Int64, degrees::Int64, Method;lambda=0)
    #  ----  Data Processing  ----
    xData = repeat(X, inner=length(X))  # Repeat xData 
    yData = repeat(Y, outer=length(Y))  # Repeat yData 
    zData = vec(Z) #Flattened Z data
    DM = DataProjector(xData, yData, zData, degrees)
    
    N = length(zData)
    split = Int(N/k_folds)
    Errors = zeros(k_folds)
    
    #Shuffle
    shuffle = sample(1:N,N,replace=false)
    xData = xData[shuffle]
    yData = yData[shuffle]
    zData = zData[shuffle]
    
    for i in 1:k_folds
        #  ----  Split Data  ----
        a = split*(i-1)+1
        b = split*(i)
        
        xTest = xData[a:b]
        yTest = yData[a:b]
        zTest = zData[a:b]
        
        xTrain = vcat(xData[1:a-1], xData[b+1:end])
        yTrain = vcat(yData[1:a-1], yData[b+1:end])
        zTrain = vcat(zData[1:a-1], zData[b+1:end])
        
        D_M_train = DataProjector(xTrain, yTrain, zTrain, degrees)
        D_M_test = DataProjector(xTest, yTest, zTest, degrees)
        
        #  ----  Apply Regression  ----
        betaVals = Regression(D_M_train, zTrain, Method, λ=lambda)
        if Method=="Lasso"
            mse=MSE(D_M_test,zTest,betaVals,λ = 0, γ = lambda)
        else
            mse = MSE(D_M_test,zTest,betaVals, λ = lambda, γ=0)
        end
        Errors[i] = mse
    end
    return mean(Errors)
end
